# Cloning the Repository (pc-gym)

In [1]:
!dir pc-gym

 Volume in drive C is Windows 
 Volume Serial Number is F0B8-32A3

 Directory of c:\Users\Usuario\Desktop\imperial_projects\VSCode\pcgym2\train\base



File Not Found


In [2]:
# %cd pc-gym

# Importing Libraries

In [3]:
# %cd src

In [4]:
# %cd pcgym

In [5]:
%cd C:/Users/Usuario/Desktop/imperial_projects/VSCode/pcgym2/pc-gym

C:\Users\Usuario\Desktop\imperial_projects\VSCode\pcgym2\pc-gym


c:\Users\Usuario\Desktop\imperial_projects\VSCode\pcgym2\new_venv\Lib\site-packages\IPython\core\magics\osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [2]:
from dataclasses import dataclass
import numpy as np
import torch
import matplotlib.pyplot as plt
import gymnasium as gym
import torch.nn.functional as F
import copy
from stable_baselines3 import PPO,SAC,DDPG,TD3
import pcgym
from pcgym import make_env
import jax.numpy as jnp
#Global params
T = 26
nsteps = 60
# Global seed for reproducibility
seed = 1990

C:\Users\mfb22\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\do_mpc\sysid\__init__.py:15: UserWarning: The ONNX feature is not available. Please install the full version of do-mpc to access this feature.
  warnings.warn('The ONNX feature is not available. Please install the full version of do-mpc to access this feature.')
C:\Users\mfb22\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\do_mpc\opcua\__init__.py:14: UserWarning: The opcua feature is not available. Please install the full version of do-mpc to access this feature.
  warnings.warn('The opcua feature is not available. Please install the full version of do-mpc to access this feature.')


# Saving and loading

Saving and loading stable-baselines models is straightforward: you can directly call `.save()` and `.load()` on the models.

In [3]:
import os
# Create save dir
save_dir = "./max/ppo"
os.makedirs(save_dir, exist_ok=True)

# Monitoring experiments with W&B

In [8]:
!wandb login 84af17cc9914cf1736f3a8e2733a2f361e4750bb

wandb: Appending key for api.wandb.ai to your netrc file: C:\Users\Usuario\.netrc


# 1.1 Reactor Case Study

In [4]:
# @title Function to log the performance data
def log_performance(performance, test_label, file_path):
    with open(file_path, "a") as file:
        file.write(f"{test_label}: \n")
        file.write(f"scalarised_performance: {performance}\n\n")

file_path = f"{save_dir}/lcb_metric_safe.txt"

### RL training

In [35]:
import numpy as np
import torch as th
from torch import nn  # Import torch.nn for activation functions

# Learning rate decay schedule
def cosine_annealing_schedule(progress_remaining: float, num_cycles=1, min_lr=0.005, max_lr=0.01):
    progress = 1.0 - progress_remaining
    lr = min_lr + (max_lr - min_lr) / 2 * (1 + np.cos(np.pi * num_cycles * progress))
    return lr

# Configuration for reinforcement learning model
config = {
    "policy": 'MlpPolicy',  # default: MlpPolicy
    "seed": 1990,
    "check_freq": 1e3,  # base: 12000 (~100 episodes)
    "n_eval_episodes": 1,  # evaluate the agent over 100 episodes in the evaluation environment
    "positive_definiteness_penalty_weight": 0,  # Set to 0 initially
    "derivative_penalty_weight": 0,  # Set to 0 initially
    'use_direct_penalty': False,  # choose between applying a penalty directly to the critic loss or adjusting the Q target values (derivative penalty)
    'allowed_increase_factor': 1,  # max increase value for both methods (derivative penalty)
    'total_timesteps': 100000,
}

# The best hyperparameters found in previous runs
best_params = {
    'min_lr': 1.790271571814614e-05,
    'max_lr': 0.021830951495112202,
    'pi_layer_0_units': 5,
    'pi_layer_1_units': 3,
    'qf_layer_0_units': 6,
    'qf_layer_1_units': 2,
    'activation_fn': 'ReLU',
    'n_steps': 2048,
    'batch_size': 128,
    'n_epochs': 11,
    'ent_coef': 2.847818037607558e-06,
    'clip_range': 0.23324165337690744,
    'gamma': 0.9095905120763823,
    'gae_lambda': 0.8795346055547795,
    'vf_coef': 0.17896972094727606,
    'max_grad_norm': 0.49101636537830917,
    'total_timesteps': 200000
}

# Update the config dictionary with the best parameters
config.update(best_params)

# Set the activation function directly based on best_params
if best_params['activation_fn'] == 'Tanh':
    activation_fn = th.nn.Tanh
elif best_params['activation_fn'] == 'ReLU':
    activation_fn = th.nn.ReLU
elif best_params['activation_fn'] == 'LeakyReLU':
    activation_fn = th.nn.LeakyReLU
else:
    raise ValueError("Unsupported activation function")

# # Create policy_kwargs with the fixed network architecture and activation function from best_params
policy_kwargs = dict(
    activation_fn=activation_fn,  # Use the activation function directly
    net_arch=dict(
        pi=[2 ** best_params['pi_layer_0_units'], 2 ** best_params['pi_layer_1_units']],
        qf=[2 ** best_params['qf_layer_0_units'], 2 ** best_params['qf_layer_1_units']]
    )
)

In [49]:
import torch as th
import numpy as np
from stable_baselines3.common.torch_layers import BaseFeaturesExtractor
from stable_baselines3.common.policies import ActorCriticPolicy
from stable_baselines3.common.vec_env import DummyVecEnv

##################################################################################
# Environment and RL Definition
##################################################################################

# Enter required setpoints for each state. Enter None for states without setpoints.
SP = {
    'T': [340.0 for _ in range(nsteps)],
}

# Continuous box action space
action_space = {
    'low': np.array([250]),
    'high': np.array([350])
}

# Continuous box observation space ([CA, T, CA_Setpoint, T_Setpoint])
observation_space = {
    'low': np.array([0.0, 300, 300]),
    'high': np.array([1, 450, 400])
}

# Define disturbance bounds
disturbance_bounds = {
    'low': np.array([340]),
    'high': np.array([360])
}

r_scale = {
    'T': 1e-6  # Reward scale for each state,
}
# Environment parameters
env_params_template = {
    'Nx': 2,
    'N': 60,
    'tsim': 26,
    'Nu': 1,
    'SP': SP,
    'o_space': observation_space,
    'a_space': action_space,
    'x0': np.array([0.87725294608097, 324.475443431599, 324.475443431599]),
    'model': 'cstr',
    'normalise_a': True,
    'normalise_o': True,
    'noise': True,
    'r_scale': r_scale,
    'integration_method': 'casadi',
    'noise_percentage': 0.001,  # 0.001,
    'disturbance_bounds': disturbance_bounds
}

# Add noise_percentage from env_params to config
config['noise_percentage'] = env_params_template['noise_percentage']

# Seed everything for reproducibility
def set_global_seeds(seed):
    np.random.seed(seed)
    th.manual_seed(seed)
    if th.cuda.is_available():
        th.cuda.manual_seed_all(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)

# Function to create random disturbances
def create_random_disturbances(seed, nsteps, low=349, high=351):
    # Set the global seed for reproducibility
    set_global_seeds(seed)
    value = np.random.uniform(low, high, 1)[0]  # Generate a single random disturbance value within the specified range
    disturbances = {'Ti': [350] * (nsteps // 3) + [value] * (nsteps // 3) + [350] * (nsteps // 3)}  # Repeat each disturbance value for nsteps/3 times
    return disturbances
# Create multiple environments with different disturbances
def create_parallel_envs(n_envs, seed):
    set_global_seeds(seed)
    envs = []
    disturbances_list = []
    for i in range(n_envs):
        env_params = env_params_template.copy()
        disturbances = create_random_disturbances(seed + i, nsteps)
        env_params.update({'disturbances': disturbances})
        disturbances_list.append(disturbances)
        envs.append(lambda: make_env(env_params))
    return DummyVecEnv(envs), disturbances_list

# Create evaluation environment using DummyVecEnv
def create_eval_env(seed, n_envs=1):
    set_global_seeds(seed)
    envs = []
    for i in range(n_envs):
        env_params = env_params_template.copy()
        disturbances = create_random_disturbances(seed + i, nsteps)
        env_params.update({'disturbances': disturbances})
        envs.append(lambda: make_env(env_params))
    return DummyVecEnv(envs)

eval_env = create_eval_env(seed=config['seed'], n_envs=1)

In [50]:

from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.vec_env import SubprocVecEnv
from stable_baselines3.common.callbacks import EvalCallback
from stable_baselines3 import PPO



# Set the global seed for reproducibility
set_global_seeds(config['seed'])

# Number of parallel environments
n_envs = 1

# Create the parallel environments and get the disturbances
env, disturbances_list = create_parallel_envs(n_envs, config['seed'])

# Print the first disturbances
print("First disturbance used in training:", disturbances_list[0])

# Training the model
model = PPO(
    config['policy'],
    env,
    learning_rate= 1e-3,#lambda progress: cosine_annealing_schedule(progress, min_lr=config['min_lr'], max_lr=config['max_lr']),  # Cosine schedule
    verbose = 1# Adding the optimized PPO-specific parameters to the model
)

# Create the evaluation environment
eval_env = create_eval_env(seed)

# Create evaluation callback

eval_callback = EvalCallback(
    eval_env,
    best_model_save_path=save_dir,
    log_path=save_dir,
    eval_freq=config['check_freq'],
    n_eval_episodes=config['n_eval_episodes'],
    deterministic=True,
    render=False
)
# Train the model with the callbacks
model.learn(total_timesteps=config['total_timesteps'],callback=eval_callback)

# Save the model
model.save(save_dir)

# Save the disturbances to a file for future reference
disturbances_file_path = os.path.join(save_dir, "disturbances_used_in_training.txt")
with open(disturbances_file_path, "w") as f:
    for i, disturbance in enumerate(disturbances_list):
        f.write(f"Disturbance {i+1}: {disturbance}\n")



First disturbance used in training: {'Ti': [350, 350, 350, 350, 350, 350, 350, 350, 350, 350, 350, 350, 350, 350, 350, 350, 350, 350, 350, 350, 350.44395451498445, 350.44395451498445, 350.44395451498445, 350.44395451498445, 350.44395451498445, 350.44395451498445, 350.44395451498445, 350.44395451498445, 350.44395451498445, 350.44395451498445, 350.44395451498445, 350.44395451498445, 350.44395451498445, 350.44395451498445, 350.44395451498445, 350.44395451498445, 350.44395451498445, 350.44395451498445, 350.44395451498445, 350.44395451498445, 350, 350, 350, 350, 350, 350, 350, 350, 350, 350, 350, 350, 350, 350, 350, 350, 350, 350, 350, 350]}
Using cuda device
Eval num_timesteps=1000, episode_reward=-0.01 +/- 0.00
Episode length: 59.00 +/- 0.00
---------------------------------
| eval/              |          |
|    mean_ep_length  | 59       |
|    mean_reward     | -0.0146  |
| time/              |          |
|    total_timesteps | 1000     |
---------------------------------
New best mean

KeyboardInterrupt: 